In [1]:
import asyncio
from ipykernel.eventloops import register_integration
import ipywidgets as widgets

In [5]:
import ccxt.async as ccxt

In [2]:
@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()
        print('Stoping Loop')

In [3]:
%gui asyncio

In [4]:
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [8]:
async def f():
    times_called = 0
    # for i in range(10):
    while True:
        # print('did work %s'%times_called)
        poloniex = ccxt.poloniex()
        ticker = await poloniex.fetch_ticker('ETH/BTC')
        # x = await wait_for_change(slider, 'value')
        # y = await wait_for_change(text, 'value')
        # a = await wait_for_change(accordion, 'value')
        # print('Slider: {}'.format(x))
        # print('Text: {}'.format(y))
        print(ticker, end='\r')
        times_called += 1
asyncio.ensure_future(f())

<Task pending coro=<f() running at <ipython-input-8-ef2e5dddd491>:1>>

In [14]:
print(times_called)

NameError: name 'times_called' is not defined

In [1]:
async def ticker_by_exchange(exchange='poloniex'):
    this_exchange = eval('ccxt.{}({})'.format(exchange))
    x = await this_exchange.fetch_tickers()
    return x

In [ ]:
for i in range(4):
    sync = asyncio.get_event_loop().run_until_complete
    # x = asyncio.get_event_loop().run_until_complete(print_poloniex_ethbtc_ticker())
    x = sync(print_poloniex_ethbtc_ticker())
    print(x)